In [1]:
# configuration

import ConfigParser
config = ConfigParser.ConfigParser()
config.read('/work/private/config')
BUNGIE_API_KEY = config.get('bungie', 'api_key')


In [2]:
import json
import requests

class BungieClient:
    def __init__(self):
        self.api_key = BUNGIE_API_KEY
        self.service_url = "http://www.bungie.net/Platform/Destiny"
        
    def get_request(self, path):
        url = self.service_url + path
        headers = {'X-API-Key': self.api_key}
        print "getting URL: " + url
        r = requests.get(url, headers=headers)
        return r.text
        
    def search_user(self, membership_type, display_name):
        path = "/SearchDestinyPlayer/%s/%s/" % (membership_type, display_name)
        response = self.get_request(path)
        result_json = json.loads(self.get_request(path))
        return result_json['Response']
    
    def get_account_details(self, membership_type, membership_id):
        path = "/%s/Account/%s/" % (membership_type, membership_id)
        response = self.get_request(path)
        result_json = json.loads(self.get_request(path))
        return result_json['Response']['data']
        #print json.dumps(result_json, sort_keys=True, indent=2)
        #return Account.from_json(result_json['Response'][0])

    def get_activity_history(self, membership_type, membership_id, character_id):
        path = "/Stats/ActivityHistory/%s/%s/%s/" % (membership_type, membership_id, character_id)
        response = self.get_request(path)
        result_json = json.loads(self.get_request(path))
        print json.dumps(result_json, sort_keys=True, indent=2)
        return result_json['Response']['data']

    def get_activity_stats(self, membership_type, membership_id, character_id):
        path = "/Stats/AggregateActivityStats/%s/%s/%s/" % (membership_type, membership_id, character_id)
        response = self.get_request(path)
        result_json = json.loads(self.get_request(path))
        print json.dumps(result_json, sort_keys=True, indent=2)
        return result_json['Response']['data']

    



In [218]:
class User:
    @classmethod
    def from_json(cls, user_json):
        return cls(user_json['membershipType'], user_json['displayName'], user_json['iconPath'], user_json['membershipId'])       
        
    def __init__(self, membership_type, display_name, icon_path, membership_id):
        self.membership_type = membership_type
        self.display_name = display_name
        self.icon_path = icon_path
        self.membership_id = membership_id


In [3]:
client = BungieClient()
users = client.search_user('All', 'cherrybomb523')
print("Got %s user(s)" % len(users))

getting URL: http://www.bungie.net/Platform/Destiny/SearchDestinyPlayer/All/cherrybomb523/
getting URL: http://www.bungie.net/Platform/Destiny/SearchDestinyPlayer/All/cherrybomb523/
Got 1 user(s)


In [5]:
for user in users:
    print user
    account_details = client.get_account_details(user['membershipType'], user['membershipId'])
    print account_details['clanName'], account_details['clanTag'], account_details['grimoireScore']
    for character in account_details['characters']:
        print character['characterLevel']
        print character['characterBase']['characterId']
        client.get_activity_history(user['membershipType'], user['membershipId'], character['characterBase']['characterId'])


{u'membershipId': u'4611686018444811938', u'displayName': u'cherrybomb523', u'membershipType': 2, u'iconPath': u'/img/theme/destiny/icons/icon_psn.png'}
getting URL: http://www.bungie.net/Platform/Destiny/2/Account/4611686018444811938/
getting URL: http://www.bungie.net/Platform/Destiny/2/Account/4611686018444811938/
DoD NA East Team Blue DoD 2260
28
2305843009294377580
getting URL: http://www.bungie.net/Platform/Destiny/Stats/ActivityHistory/2/4611686018444811938/2305843009294377580/
getting URL: http://www.bungie.net/Platform/Destiny/Stats/ActivityHistory/2/4611686018444811938/2305843009294377580/
{
  "ErrorCode": 19, 
  "ErrorStatus": "ParameterNotFound", 
  "Message": "We've encountered an error, please try again later.", 
  "MessageData": {}, 
  "ThrottleSeconds": 0
}


KeyError: 'Response'